In [51]:
import pandas as pd
import numpy as np
import datetime as dt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [3]:
raw_data = pd.read_csv("RawData.csv")

In [17]:
raw_data['Date'] = pd.to_datetime(raw_data['Date'])


In [18]:
raw_data.head()

,Date,Name,Team,Season,Round,Home Team,Away Team,Home Score,Away Score,Margin,...,Frees For,Frees Against,Contested Pos,Uncontested Pos,Contested Marks,Marks Inside 50,One Percenters,Goal Assists,Brownlow Votes,TOG
0,2019-09-28,Zac Williams,Greater Western Sydney,2019.0,GF,Richmond,Greater Western Sydney,114,25,89,...,0.0,0.0,7.0,13.0,0.0,1.0,0.0,0.0,0.0,68
1,2019-09-28,Lachie Whitfield,Greater Western Sydney,2019.0,GF,Richmond,Greater Western Sydney,114,25,89,...,0.0,1.0,4.0,9.0,0.0,0.0,1.0,0.0,0.0,91
2,2019-09-28,Adam Tomlinson,Greater Western Sydney,2019.0,GF,Richmond,Greater Western Sydney,114,25,89,...,0.0,1.0,6.0,11.0,0.0,0.0,1.0,0.0,0.0,83
3,2019-09-28,Sam Taylor,Greater Western Sydney,2019.0,GF,Richmond,Greater Western Sydney,114,25,89,...,1.0,0.0,6.0,8.0,1.0,0.0,9.0,0.0,0.0,91
4,2019-09-28,Tim Taranto,Greater Western Sydney,2019.0,GF,Richmond,Greater Western Sydney,114,25,89,...,3.0,1.0,15.0,18.0,0.0,0.0,1.0,0.0,0.0,80


In [79]:
#training_new = raw_data[raw_data['Date'].dt.year.isin(range(2013, 2016))]
training_new = raw_data[raw_data['Date'].dt.year.isin([2013])]

In [67]:
test_data = raw_data[raw_data['Date'].dt.year.isin(range(2016, 2020))]

In [123]:
df = pd.DataFrame(np.random.randn(8, 4), columns=['A', 'B', 'C', 'D'])
df.head()

,A,B,C,D
0,0.757257,0.073601,0.078423,-0.548408
1,-1.363076,0.735929,1.160571,-0.054366
2,-0.031771,1.094653,0.366450,0.530213
3,0.318950,0.010752,0.270199,0.481996
4,0.409486,0.386002,0.100510,0.750016


In [96]:
LogReg = LogisticRegression(solver='sag',max_iter = 5000)
training_data_new = training_data.drop("Date", axis = 1)
training_data_new = training_data_new.drop("Name", axis = 1)
training_data_new = training_data_new.drop("Team", axis = 1)
training_data_new = training_data_new.drop("Round", axis = 1)
training_data_new = training_data_new.drop("Home Team", axis = 1)
training_data_new = training_data_new.drop("Away Team", axis = 1)
LogReg.fit(training_data_new.drop("Brownlow Votes",axis = 1) , training_data_new["Brownlow Votes"])
#LogReg.fit(training_data_new[["Disposals", "Goals"]] , training_data_new["Brownlow Votes"])

#model1 = linear_model.LogisticRegressionCV(max_iter = 4000) 
#LogReg2= model1(solver= 'sag') 

LogisticRegression(max_iter=5000, solver='sag')

In [143]:
test_data_new = test_data.drop("Date", axis = 1)
test_data_new = test_data_new.drop("Name", axis = 1)
test_data_new = test_data_new.drop("Team", axis = 1)
test_data_new = test_data_new.drop("Round", axis = 1)
test_data_new = test_data_new.drop("Home Team", axis = 1)
test_data_new = test_data_new.drop("Away Team", axis = 1)
test_data_new = test_data_new.drop("Brownlow Votes", axis = 1)
#test_data_1 = test_data[["Disposals", "Goals"]]
res = LogReg.predict(test_data_new)
#print(len(res))
#print(len(test_data['Brownlow Votes']))
actual_all = list(test_data['Brownlow Votes'])
predicted_all = list(res)
counter = 0
for i in range(len(actual_all)):
    if actual_all[i] == predicted_all[i]:
        counter +=1
print(counter)
print(len(actual_all))
diff = counter/len(actual_all) * 100
print(diff)

34303
36432
94.15623627580149
